In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 65.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55

In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [6]:
DOCUMENT1 = "AI in Medical Imaging and Disease Diagnosis  Artificial Intelligence (AI) is transforming medical imaging and disease diagnosis by providing faster, more accurate analysis of medical scans. Traditional diagnostic methods often rely on human interpretation, which can be time-consuming and subject to human error. AI-powered medical imaging solutions use deep learning algorithms trained on vast datasets of X-rays, MRIs, and CT scans to identify abnormalities such as tumors, fractures, and infections with high accuracy.  One significant application of AI in medical imaging is the early detection of cancers, such as lung and breast cancer. AI models have been trained to detect subtle patterns in mammograms and CT scans that might be missed by radiologists. In one study, AI outperformed human radiologists in detecting breast cancer, reducing false positives and minimizing unnecessary biopsies.  AI also plays a critical role in neuroimaging, assisting in the diagnosis of conditions like Alzheimer's disease, stroke, and multiple sclerosis. Deep learning models analyze MRI scans to detect early signs of cognitive decline, enabling early interventions and better patient outcomes.  Another breakthrough in AI-driven medical imaging is its use in ophthalmology. AI-powered retinal imaging systems can detect diabetic retinopathy, a leading cause of blindness, by analyzing retinal scans for signs of damage. This allows for early treatment and reduces the risk of vision loss.  Overall, AI in medical imaging enhances diagnostic accuracy, speeds up workflow, and provides valuable insights for doctors. With continuous advancements in deep learning and computer vision, AI-driven imaging tools are expected to become an integral part of clinical practice."

DOCUMENT2 = "AI in Predictive Analytics and Personalized Medicine  Predictive analytics in healthcare uses AI to analyze vast amounts of patient data and identify patterns that indicate potential health risks. AI-driven predictive models process electronic health records (EHRs), genetic information, and lifestyle data to forecast the likelihood of diseases such as diabetes, heart disease, and Alzheimer’s. This allows for early interventions, personalized treatment plans, and improved patient outcomes.  One major area where AI is making a difference is in cardiology. AI models trained on electrocardiograms (ECGs) and echocardiograms can detect early signs of heart disease, including arrhythmias and heart failure, before symptoms appear. By analyzing a patient’s medical history and risk factors, AI can predict the probability of a heart attack, allowing doctors to take preventive measures.  In oncology, AI is revolutionizing cancer treatment by enabling personalized medicine. AI algorithms analyze a patient’s genetic profile to determine the most effective cancer treatment, reducing trial-and-error approaches and improving survival rates. For example, AI-driven precision oncology tools match cancer patients with targeted therapies based on their unique genetic mutations.  AI also plays a role in predicting disease outbreaks and pandemics. By analyzing global health data, AI models can identify patterns in the spread of infectious diseases, enabling governments and healthcare organizations to implement timely interventions. For instance, AI was used to track and predict the spread of COVID-19 by analyzing data from online reports, travel patterns, and hospital records.  Overall, AI-driven predictive analytics enhances early disease detection, enables precision medicine, and optimizes healthcare delivery. With continuous improvements in AI models, healthcare providers can make more informed decisions, ultimately leading to better patient care and outcomes."

DOCUMENT3 = "AI in Drug Discovery and Clinical Trials  The process of discovering new drugs is traditionally slow, costly, and requires years of research and testing. AI is transforming drug discovery by analyzing massive datasets, predicting potential drug candidates, and optimizing clinical trials, significantly accelerating the process.  AI-driven drug discovery involves machine learning models that analyze chemical structures and predict how they interact with biological systems. By screening millions of compounds in silico, AI helps researchers identify promising drug candidates much faster than traditional methods. For example, AI was instrumental in identifying potential treatments for COVID-19 by analyzing existing drugs that could be repurposed.  AI is also revolutionizing clinical trials by optimizing patient recruitment and improving trial efficiency. One of the major challenges in clinical trials is finding suitable participants who meet specific criteria. AI-driven tools analyze electronic health records and genetic data to identify the best candidates, ensuring trials are conducted with the most relevant patient groups. This reduces the time and cost associated with traditional recruitment methods.  Additionally, AI enhances the monitoring of clinical trials by analyzing real-time patient data. Wearable devices and sensors collect continuous health data, which AI processes to identify potential adverse effects or treatment responses. This enables researchers to make data-driven adjustments to trials, ensuring patient safety and improving outcomes.  Beyond drug discovery and trials, AI is used in precision medicine to develop personalized treatment plans. By analyzing a patient’s genetic makeup, lifestyle, and medical history, AI can recommend the most effective drugs with minimal side effects. This approach has been particularly beneficial in cancer treatment, where targeted therapies are designed based on a patient’s unique tumor profile.  In summary, AI accelerates drug discovery, optimizes clinical trials, and enables precision medicine. As AI continues to advance, the pharmaceutical industry will benefit from faster, more efficient drug development, ultimately leading to better treatments for patients."


documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [11]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/gemini-embedding-exp-03-07",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

In [12]:
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

In [13]:
db.count()
# You can peek at the data too.
# db.peek(1)

3

In [14]:
db.peek(1)

{'ids': ['0'],
 'embeddings': array([[-0.00785584,  0.00185402, -0.00313931, ..., -0.01716148,
         -0.0067208 , -0.00444444]]),
 'documents': ["AI in Medical Imaging and Disease Diagnosis  Artificial Intelligence (AI) is transforming medical imaging and disease diagnosis by providing faster, more accurate analysis of medical scans. Traditional diagnostic methods often rely on human interpretation, which can be time-consuming and subject to human error. AI-powered medical imaging solutions use deep learning algorithms trained on vast datasets of X-rays, MRIs, and CT scans to identify abnormalities such as tumors, fractures, and infections with high accuracy.  One significant application of AI in medical imaging is the early detection of cancers, such as lung and breast cancer. AI models have been trained to detect subtle patterns in mammograms and CT scans that might be missed by radiologists. In one study, AI outperformed human radiologists in detecting breast cancer, reducing fal

In [15]:
#Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "How does AI help in diagnosing diseases?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

AI in Medical Imaging and Disease Diagnosis  Artificial Intelligence (AI) is transforming medical imaging and disease diagnosis by providing faster, more accurate analysis of medical scans. Traditional diagnostic methods often rely on human interpretation, which can be time-consuming and subject to human error. AI-powered medical imaging solutions use deep learning algorithms trained on vast datasets of X-rays, MRIs, and CT scans to identify abnormalities such as tumors, fractures, and infections with high accuracy.  One significant application of AI in medical imaging is the early detection of cancers, such as lung and breast cancer. AI models have been trained to detect subtle patterns in mammograms and CT scans that might be missed by radiologists. In one study, AI outperformed human radiologists in detecting breast cancer, reducing false positives and minimizing unnecessary biopsies.  AI also plays a critical role in neuroimaging, assisting in the diagnosis of conditions like Alzheimer's disease, stroke, and multiple sclerosis. Deep learning models analyze MRI scans to detect early signs of cognitive decline, enabling early interventions and better patient outcomes.  Another breakthrough in AI-driven medical imaging is its use in ophthalmology. AI-powered retinal imaging systems can detect diabetic retinopathy, a leading cause of blindness, by analyzing retinal scans for signs of damage. This allows for early treatment and reduces the risk of vision loss.  Overall, AI in medical imaging enhances diagnostic accuracy, speeds up workflow, and provides valuable insights for doctors. With continuous advancements in deep learning and computer vision, AI-driven imaging tools are expected to become an integral part of clinical practice.

In [16]:
#Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "Can AI help in clinical trials?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

AI in Drug Discovery and Clinical Trials  The process of discovering new drugs is traditionally slow, costly, and requires years of research and testing. AI is transforming drug discovery by analyzing massive datasets, predicting potential drug candidates, and optimizing clinical trials, significantly accelerating the process.  AI-driven drug discovery involves machine learning models that analyze chemical structures and predict how they interact with biological systems. By screening millions of compounds in silico, AI helps researchers identify promising drug candidates much faster than traditional methods. For example, AI was instrumental in identifying potential treatments for COVID-19 by analyzing existing drugs that could be repurposed.  AI is also revolutionizing clinical trials by optimizing patient recruitment and improving trial efficiency. One of the major challenges in clinical trials is finding suitable participants who meet specific criteria. AI-driven tools analyze electronic health records and genetic data to identify the best candidates, ensuring trials are conducted with the most relevant patient groups. This reduces the time and cost associated with traditional recruitment methods.  Additionally, AI enhances the monitoring of clinical trials by analyzing real-time patient data. Wearable devices and sensors collect continuous health data, which AI processes to identify potential adverse effects or treatment responses. This enables researchers to make data-driven adjustments to trials, ensuring patient safety and improving outcomes.  Beyond drug discovery and trials, AI is used in precision medicine to develop personalized treatment plans. By analyzing a patient’s genetic makeup, lifestyle, and medical history, AI can recommend the most effective drugs with minimal side effects. This approach has been particularly beneficial in cancer treatment, where targeted therapies are designed based on a patient’s unique tumor profile.  In summary, AI accelerates drug discovery, optimizes clinical trials, and enables precision medicine. As AI continues to advance, the pharmaceutical industry will benefit from faster, more efficient drug development, ultimately leading to better treatments for patients.

In [17]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative AI bot specialised in healthcare that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience and provide advise based on your understanding of the refernce above ., so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

You are a helpful and informative AI bot specialised in healthcare that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience and provide advise based on your understanding of the refernce above ., so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: Can AI help in clinical trials?
PASSAGE: AI in Drug Discovery and Clinical Trials  The process of discovering new drugs is traditionally slow, costly, and requires years of research and testing. AI is transforming drug discovery by analyzing massive datasets, predicting potential drug candidates, and optimizing clinical trials, significantly accelerating the process.  AI-driven drug discovery involves machine learning models that analyze chemical 

In [19]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

Yes, AI can be a big help in clinical trials by making the process faster and more efficient! One of the main ways AI does this is by helping to find the right patients for trials by analyzing electronic health records and genetic data to pinpoint the best candidates who meet the trial's specific requirements, which reduces the time and money spent on traditional recruitment methods. Additionally, AI can keep a close eye on clinical trials by analyzing real-time patient data collected from wearable devices and sensors, helping researchers spot any potential adverse effects or treatment responses quickly, allowing for adjustments to be made to ensure patient safety and improve the trial's success.


In [21]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 33.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 76.5 MB/s eta 0:00:00:00:0100:01


In [23]:
import gradio as gr

def ai_healthcare_bot(user_query):
    # Search the Chroma DB using the user's query
    result = db.query(query_texts=[user_query], n_results=1)
    [all_passages] = result["documents"]

    # Format the response
    query_oneline = user_query.replace("\n", " ")
    prompt = f"""You are a helpful AI bot specialized in healthcare.
    Answer using text from the reference passage below. 
    If the passage is irrelevant, provide a general AI-based answer.

    QUESTION: {query_oneline}
    """

    for passage in all_passages:
        passage_oneline = passage.replace("\n", " ")
        prompt += f"PASSAGE: {passage_oneline}\n"

    # Generate AI response using Gemini
    answer = client.models.generate_content(model="gemini-2.0-flash", contents=prompt)

    return answer.text  # Return the generated response

# Create Gradio interface
interface = gr.Interface(
    fn=ai_healthcare_bot,
    inputs=gr.Textbox(label="Ask me about AI in Healthcare"),
    outputs="text",
    title="AI Healthcare Chatbot",
    description="Ask any question about AI in healthcare, and I will provide an AI-generated response!"
)

# Launch the app in Kaggle
interface.launch(share=True)  

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://67a438aaeacc3e988a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Created dataset file at: .gradio/flagged/dataset1.csv
